In [11]:
import pandas as pd
import requests, json, os

def get_data(pids, years):
    id = 'usnavy09'
    apikey = 'J0RL197TS3DBGG61VRPDAF8AECQXM7AU07DVZS8NY553P'
    url = 'http://www.recycling-info.or.kr/sds/JsonApi.do?'
    items = []
    
    # pid, year 반복
    for pid in pids:
        for year in years:
            param = f'PID={pid}&YEAR={year}&USRID={id}&KEY={apikey}'
            
            try:
                response = requests.get(url=url + param).json()
                # 예외처리
                if 'error' in response:
                    raise Exception(f"API 오류: {response['error']}")
                elif response['result'][0]['ERR_CODE'] != 'E000':
                    raise Exception(f"error : {response['result'][0]['RESULT']}")
                
                # 집계된 데이터 제외
                for item in response['data']:
                    if '합계' not in item.values():
                        items.append(item)
                
                # 컬럼을 한글로 바꾼 후 데이터 저장
                year_label = response['result'][0]['YEAR']
                title_label = response['result'][0]['TITLE']
                df = pd.json_normalize(items)
                df.rename(columns=response['dataHeader'][0], inplace=True)
                
                # data-files 폴더 생성
                if 'data-files' not in os.listdir():
                    os.makedirs('data-files')
                df.to_csv(f'data-files/{year_label}_{title_label}.csv', index=False)
                print(f"데이터 저장완료 : {year_label}_{title_label}.csv")
                
            except requests.exceptions.RequestException as e:
                print(f"네트워크 오류: {e}")
                
            except Exception as e:
                print(f"예외 발생: {e}")
        

In [12]:
pids = ['NTN001', 'NTN002']
years = [2016, 2017]
get_data(pids, years)

데이터 저장완료 : 2016_1가(생활폐기물 관리구역).csv
데이터 저장완료 : 2017_1가(생활폐기물 관리구역).csv
데이터 저장완료 : 2016_1나(사업장폐기물배출업소).csv
데이터 저장완료 : 2017_1나(사업장폐기물배출업소).csv
